In [ ]:
!pip install tensorflow tensorflow_data_validation tensorflow_transform ml-metadata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow-serving-api to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
import os
import requests
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
from ml_metadata.metadata_store import metadata_store


In [ ]:
FULL_DATASET = '../covertype.csv'
SMALL_DATASET= '../covertype_small.csv'
TRAINING_DATASET='../covertype_training.csv'
TRAINING_DATASET_WITH_MISSING = '../covertype_training_missing.csv'
EVALUATION_DATASET='../covertype_evaluation.csv'
EVALUATION_DATASET_WITH_ANOMALIES='../covertype_evaluation_anomalies.csv'
SERVING_DATASET='../covertype_serving.csv'

ORIGINAL_DATASET_PATH = 'gs://workshop-datasets/covertype/orig/covtype.data'

In [ ]:
## download the dataset
# Directory of the raw data files
_data_root = './data/covertype'

In [ ]:
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')

In [ ]:
# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
    url = 'https://docs.google.com/uc?export= \
    download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)

In [ ]:
file_path = "./data/covertype/covertype_train.csv"  # Reemplazar con la ruta correcta
df = pd.read_csv(file_path)

In [ ]:
columns = df.columns.tolist()
columns

['Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area',
 'Soil_Type',
 'Cover_Type']

In [ ]:
# Selección de características
features = ['Elevation','Aspect','Hillshade_3pm', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
            'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Horizontal_Distance_To_Fire_Points']
target = 'Cover_Type'

X = df[features]
y = df[target]

In [ ]:
# Convertir variables categóricas a numéricas con One-Hot Encoding
X = pd.get_dummies(X, drop_first=True)
X

,Elevation,Aspect,Hillshade_3pm,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points
0,2991,119,133,7,67,11,1015,233,234,1570
1,2876,3,144,18,485,71,2495,192,202,1557
2,3171,315,162,2,277,9,4374,213,237,1052
3,3087,342,166,13,190,31,4774,193,221,752
4,2835,158,141,10,212,41,3596,231,242,3280
...,...,...,...,...,...,...,...,...,...,...
116198,3150,220,187,16,285,47,2275,200,253,866
116199,3125,47,120,13,234,2,2430,224,212,1426
116200,3166,152,136,11,67,0,1275,234,240,2404
116201,3154,285,198,14,738,46,6012,181,239,1320


In [ ]:
selector = SelectKBest(score_func=f_classif, k=10)
X_selected = selector.fit_transform(X, y)
selected_feature_indices = selector.get_support(indices=True)
selected_features = X.columns[selected_feature_indices]
print("Mejores características seleccionadas:", selected_features)

Mejores características seleccionadas: Index(['Elevation', 'Aspect', 'Hillshade_3pm', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Horizontal_Distance_To_Fire_Points'],
      dtype='object')


In [ ]:
# Normalización de datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

In [ ]:
# Creación del pipeline con TFX
stats = tfdv.generate_statistics_from_dataframe(df)
tfdv.visualize_statistics(stats)

In [ ]:
schema = tfdv.infer_schema(stats)
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Elevation',INT,required,,-
'Aspect',INT,required,,-
'Slope',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Vertical_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Hillshade_3pm',INT,required,,-


,Values
Domain,
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"


In [ ]:
dataset_anomalies = tfdv.validate_statistics(stats, schema)
tfdv.display_anomalies(dataset_anomalies)

In [ ]:
# Transformación de datos
def preprocessing_fn(inputs):
    outputs = {}
    outputs['Elevation'] = tft.scale_to_z_score(inputs['Elevation'])
    outputs['Slope'] = tft.scale_to_0_1(inputs['Slope'])
    outputs['Cover_Type'] = inputs['Cover_Type']
    return outputs

In [ ]:
# Función para asignar un dominio si no existe
def set_domain_if_missing(schema, feature_name, min_val, max_val):
    feature = tfdv.get_feature(schema, feature_name)
    if feature.domain is None:
        feature.int_domain.CopyFrom(schema_pb2.IntDomain(min=min_val, max=max_val))

# Ajustar los dominios de valores esperados
set_domain_if_missing(schema, 'Hillshade_9am', 0, 255)
set_domain_if_missing(schema, 'Hillshade_Noon', 0, 255)
set_domain_if_missing(schema, 'Slope', 0, 90)
set_domain_if_missing(schema, 'Cover_Type', 0, 6)

# Mostrar el esquema curado
tfdv.display_schema(schema)
print("Esquema curado correctamente.")

,Type,Presence,Valency,Domain
Feature name,,,,
'Elevation',INT,required,,-
'Aspect',INT,required,,-
'Slope',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Vertical_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Hillshade_3pm',INT,required,,-


,Values
Domain,
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"


Esquema curado correctamente.


In [ ]:


print("Pipeline completado exitosamente.")

Pipeline completado exitosamente.
